PREPROCESSING

In [41]:
import numpy as np
from numpy.random import normal
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
from scipy.interpolate import BSpline
from cmdstanpy import CmdStanModel
import arviz as az
import pandas as pd
from glob import glob
from sklearn import preprocessing
import csv

In [42]:
def load_signals (files):
    signals  = np.zeros((len(files),), dtype = object)
    for index,i in enumerate(files):
        file = np.loadtxt(i, dtype = 'str' , unpack = True)
        file = np.char.replace(file,',','.')
        file = file.astype(np.float64)
        signals[index] = file
    return signals

def filtr_1(signal):
    index = 0
    y = 0

   
    while (y < 0.01):
        y = signal[1][index]
        index += 1

    index -= 2
    signal_y = signal[1][index:]
    signal_x = signal[0][index:]
    trunc_val_peak = -2.5
    signal_y = signal_y[signal_y > trunc_val_peak]

    index = 1
    while abs(signal_y[-index] - signal_y[-(index+1)]) < 0.02:
        index += 1

    signal_y = signal_y[:-index]
    signal_x = signal_x[:len(signal_y)]

    return signal_x,signal_y

def filtr_2(signal_x, signal_y):
    
    cum_sum = np.cumsum(signal_y[::-1])[::-1]
    cum_mean = np.flip(cum_sum) / np.arange(1,len(cum_sum)+1)

    index = 1
    while abs(signal_y[-index-1] - cum_mean[index]) < 0.02:
        index += 1
    
    index -= 2
    signal_y_filtered = signal_y[:-index]
    signal_x_filtered = signal_x[:len(signal_y_filtered)]
    
    return signal_x_filtered,signal_y_filtered

def rms(y,x):
    rms = []
    for i in range(len(x)-1):
        rms = np.append(rms,(np.trapz(np.square(y[i:2+i]),x[i:2+i])))
    rms = np.sqrt(np.sum(rms)/(max(x)- min(x)))
    return rms
    

    
def rms_signals(signals_x,signals_y):
        num_per_rms = 23
        num = np.ceil(len(signals_y)/num_per_rms)
        y = np.array_split(signals_y,num)
        time = np.array_split(signals_x,num)
        signal_rms = []
        time_rms = []
        for i in range(len(y)):
            signal_rms =np.append(signal_rms,rms(y[i],time[i]))
            time_rms = np.append(time_rms, np.mean(time[i]))
    
        return time_rms, signal_rms



min_max_scaler = preprocessing.MinMaxScaler()

In [184]:
files_healthy = glob('data/p0_*.*')
files_damaged_1 = glob('data/p1_*.*')
files_damaged_2 = glob('data/p2_*.*')
files_damaged_ring = glob('data/pi_*.*')

signals_healthy = load_signals(files_healthy)
signals_damaged_1 = load_signals(files_damaged_1)
signals_damaged_2 = load_signals(files_damaged_2)
signals_damaged_ring = load_signals(files_damaged_ring)

In [183]:
files_healthy

[]

In [44]:
signals_healthy_x = np.zeros((len(signals_healthy),), dtype = object)
signals_healthy_y = np.zeros((len(signals_healthy),), dtype = object)

for index,i in enumerate(signals_healthy):
    signals_healthy_x[index],signals_healthy_y[index] = filtr_1(i)
    signals_healthy_x[index],signals_healthy_y[index] = rms_signals(signals_healthy_x[index],signals_healthy_y[index]) 
    signals_healthy_x[index],signals_healthy_y[index] = filtr_2(signals_healthy_x[index],signals_healthy_y[index])
    # signals_healthy_x[index] = min_max_scaler.fit_transform(signals_healthy_x[index].reshape(-1, 1)).astype(float)    #normalization
    # signals_healthy_y[index] = min_max_scaler.fit_transform(signals_healthy_y[index].reshape(-1, 1)).astype(float)    #normalization
    signals_healthy_x[index] = signals_healthy_x[index] - min(signals_healthy_x[index])

In [45]:
signals_damaged_1_x = np.zeros((len(signals_damaged_1),), dtype = object)
signals_damaged_1_y = np.zeros((len(signals_damaged_1),), dtype = object)

for index,i in enumerate(signals_damaged_1):
    signals_damaged_1_x[index],signals_damaged_1_y[index] = filtr_1(i)
    signals_damaged_1_x[index],signals_damaged_1_y[index] = rms_signals(signals_damaged_1_x[index],signals_damaged_1_y[index]) 
    signals_damaged_1_x[index],signals_damaged_1_y[index] = filtr_2(signals_damaged_1_x[index],signals_damaged_1_y[index])
    # signals_damaged_1_x[index] = min_max_scaler.fit_transform(signals_damaged_1_x[index].reshape(-1, 1)).astype(float)    #normalization        
    # signals_damaged_1_y[index] = min_max_scaler.fit_transform(signals_damaged_1_y[index].reshape(-1, 1)).astype(float)    #normalization    
    signals_damaged_1_x[index] = signals_damaged_1_x[index] - min(signals_damaged_1_x[index])

In [46]:
signals_damaged_2_x = np.zeros((len(signals_damaged_2),), dtype = object)
signals_damaged_2_y = np.zeros((len(signals_damaged_2),), dtype = object)

for index,i in enumerate(signals_damaged_2):
    signals_damaged_2_x[index],signals_damaged_2_y[index] = filtr_1(i)
    signals_damaged_2_x[index],signals_damaged_2_y[index] = rms_signals(signals_damaged_2_x[index],signals_damaged_2_y[index]) 
    signals_damaged_2_x[index],signals_damaged_2_y[index] = filtr_2(signals_damaged_2_x[index],signals_damaged_2_y[index])
    # signals_damaged_2_x[index] = min_max_scaler.fit_transform(signals_damaged_2_x[index].reshape(-1, 1)).astype(float)    #normalization
    # signals_damaged_2_y[index] = min_max_scaler.fit_transform(signals_damaged_2_y[index].reshape(-1, 1)).astype(float)    #normalization
    signals_damaged_2_x[index] = signals_damaged_2_x[index] - min(signals_damaged_2_x[index])

In [47]:
signals_damaged_ring_x = np.zeros((len(signals_damaged_ring),), dtype = object)
signals_damaged_ring_y = np.zeros((len(signals_damaged_ring),), dtype = object)

for index,i in enumerate(signals_damaged_ring):
    signals_damaged_ring_x[index],signals_damaged_ring_y[index] = filtr_1(i)
    signals_damaged_ring_x[index],signals_damaged_ring_y[index] = rms_signals(signals_damaged_ring_x[index],signals_damaged_ring_y[index]) 
    signals_damaged_ring_x[index],signals_damaged_ring_y[index] = filtr_2(signals_damaged_ring_x[index],signals_damaged_ring_y[index])
    # signals_damaged_ring_x[index] = min_max_scaler.fit_transform(signals_damaged_ring_x[index].reshape(-1, 1)).astype(float)  #normalization
    # signals_damaged_ring_y[index] = min_max_scaler.fit_transform(signals_damaged_ring_y[index].reshape(-1, 1)).astype(float)  #normalization
    signals_damaged_ring_x[index] = signals_damaged_ring_x[index] - min(signals_damaged_ring_x[index])

In [48]:
for i in range(len(signals_healthy_y)):
    signals_healthy_y[i] = signals_healthy_y[i].reshape(len(signals_healthy_y[i]))

for i in range(len(signals_damaged_1_y)):
    signals_damaged_1_y[i] = signals_damaged_1_y[i].reshape(len(signals_damaged_1_y[i]))

for i in range(len(signals_damaged_2_y)):
    signals_damaged_2_y[i] = signals_damaged_2_y[i].reshape(len(signals_damaged_2_y[i]))

for i in range(len(signals_damaged_ring_y)):
    signals_damaged_ring_y[i] = signals_damaged_ring_y[i].reshape(len(signals_damaged_ring_y[i]))

In [179]:
for i in range(len(signals_healthy)):
    data = pd.DataFrame((signals_healthy_y[i],signals_healthy_x[i]))
    data.to_csv('processed_data1/signals_healthy_{}.csv'.format(i), index=False)

for i in range(len(signals_damaged_1)):
    data = pd.DataFrame((signals_damaged_1_y[i],signals_damaged_1_x[i]))
    data.to_csv('processed_data1/signals_damaged_1_{}.csv'.format(i), index=False)

for i in range(len(signals_damaged_2)):
    data = pd.DataFrame((signals_damaged_2_y[i],signals_damaged_2_x[i]))
    data.to_csv('processed_data1/signals_damaged_2_{}.csv'.format(i), index=False)

for i in range(len(signals_damaged_ring)):
    data = pd.DataFrame((signals_damaged_ring_y[i],signals_damaged_ring_x[i]))
    data.to_csv('processed_data1/signals_damaged_ring_{}.csv'.format(i), index=False)

In [191]:
signals_damaged_ring_x22 = np.zeros((7,), dtype = object)
signals_damaged_ring_y22 = np.zeros((7,), dtype = object)
for i in range(7):
    a = pd.read_csv('processed_data1/signals_damaged_2_{}.csv'.format(i)) 
    signals_damaged_ring_y22[i] = a.to_numpy()[0]
    signals_damaged_ring_x22[i] = a.to_numpy()[1]

In [195]:
signals_damaged_ring_x22[0]

array([0.      , 0.019987, 0.039974, 0.059961, 0.079948, 0.099935,
       0.119922, 0.139909, 0.159896, 0.179883, 0.19987 , 0.219857,
       0.239844, 0.259831, 0.279818, 0.299805, 0.319792, 0.339779,
       0.359766, 0.379753, 0.39974 , 0.419727, 0.439714, 0.459701,
       0.479688, 0.499675, 0.519662, 0.539649, 0.559636, 0.579623,
       0.59961 , 0.619597, 0.639584, 0.659571, 0.679558, 0.699545,
       0.719532, 0.739519, 0.759506, 0.779493, 0.79948 , 0.819467,
       0.839454, 0.859441, 0.879428, 0.899415, 0.919402, 0.939389,
       0.959376, 0.979363, 0.99935 , 1.019337, 1.039324, 1.059311,
       1.079298, 1.099285, 1.119272, 1.139259, 1.159246, 1.179233,
       1.19922 , 1.219207, 1.239194, 1.259181, 1.279168, 1.299155,
       1.319142, 1.339129, 1.359116, 1.379103, 1.39909 , 1.419077,
       1.439064, 1.459051, 1.479038, 1.499025, 1.519012, 1.538999,
       1.558986, 1.578973, 1.59896 , 1.618947, 1.638934, 1.658921,
       1.678908, 1.698895, 1.718882, 1.738869, 1.758856, 1.778